In [ ]:
# Import required packages
import sys
# !{sys.executable} -m pip install seaborn
from glob import glob
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from shutil import copy,move
from distutils.dir_util import copy_tree


In [ ]:
def results_df(path):
    '''
    Take the top directory where the results are and return a nice Pandas dataframe

    parameters
    ----------
    path: str
        Path to the top directory `path/*/*results.json`

    returns
    -------
    df: pd.DataFrame
        Dataframe containing results
    '''
    results_dict = {}
    for i in glob(os.path.join(path,'*/*results.json')):
        with open(i,'r') as f:
            results = json.load(f)
        name = os.path.basename(i).replace('_results.json','')
        results['encoder']=name.split('_')[0]
        results['ansatz']=name.split('_')[1]
        results_dict[name]=results

    df = pd.DataFrame(results_dict).T
    
    return df



In [ ]:
def plot_all(df,name,save_dir):
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(12,6), gridspec_kw={'width_ratios': [1, 1]})
    SMALL_SIZE = 8
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 12
    figsize = fig.get_size_inches()
    fig.set_size_inches(figsize * 1.2)
    plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
    sns.boxplot(data=df,x='encoder',y='value',hue='variable',ax=ax1,palette=sns.color_palette('Paired',2))
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90) 
    ax1.set_ylim(-1.01,1.01)
    ax1.set_ylabel('R$^{2}$')
    ax1.set_xlabel('Encoder')
    ax1.legend(title='')
    
    sns.boxplot(data=df,x='ansatz',y='value',hue='variable',ax=ax2,palette=sns.color_palette('Paired',2))
    ax2.set_xticklabels(ax2.get_xticklabels(), rotation=90) 
    ax2.set_ylim(-1.01,1.01)
    ax2.set_ylabel('R$^{2}$')
    ax2.set_xlabel('Ansatz')
    ax2.legend(title='')
    plt.tight_layout()
    plt.savefig(os.path.join(os.path.expanduser('~'),save_dir,f'{name}_boxplots.png'),dpi=300,bbox_inches='tight')
    plt.show()
    
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(13,6),sharey=False)
    SMALL_SIZE = 8
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 12
    
    plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
    figsize = fig.get_size_inches()
    fig.set_size_inches(figsize * 1.5)
    ax1=sns.heatmap(df[df['variable']=='Train'][["encoder", "ansatz", "value"]].pivot(index='encoder', columns='ansatz', values='value').astype(float),annot_kws={"size": 12}, square=True, annot=True, linewidth=.5,fmt='.2f',ax=ax1,vmax=1,vmin=-1,cbar=True,center=0,cmap='rocket')
    ax1.set_title("Train")
    ax1.set_ylabel('Encoder')
    ax1.set_xlabel('Ansatz')
    ax1.set_aspect("equal")
    
    ax2=sns.heatmap(df[df['variable']=='Test'][["encoder", "ansatz", "value"]].pivot(index='encoder', columns='ansatz', values='value').astype(float), annot_kws={"size": 12}, square=True, annot=True, linewidth=.5,fmt='.2f',ax=ax2,vmax=1,vmin=-1,center=0,cmap='rocket')
    ax2.set_title("Test")
    ax2.set_ylabel('Encoder')
    ax2.set_xlabel('Ansatz')
    ax2.set_aspect("equal")
    
    
    plt.subplots_adjust(wspace=0.1,hspace=0)
    plt.tight_layout()  # Leave space for color bar
    plt.savefig(os.path.join(os.path.expanduser('~'),save_dir,f'{name}_heatplots.png'),dpi=300,bbox_inches='tight')
    plt.show()

In [ ]:
def plot(data_dir,name,save_dir):
    df = results_df(data_dir).rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars=['encoder','ansatz'], value_vars=['Train','Test']).sort_values(by=['encoder','ansatz'])
    plot_all(df,name,save_dir)    

In [ ]:
plot("function-calc-test/sixteenqubit_test/lin16qubit/",'linear','qregress/images/Function_Fitting/sixteenqubit/')
plot("function-calc-test/sixteenqubit_test/quad16qubit/",'quadratic','qregress/images/Function_Fitting/sixteenqubit/')
plot("function-calc-test/sixteenqubit_test/sin16qubit/",'sine','qregress/images/Function_Fitting/sixteenqubit/')

In [ ]:
plot("function-calc-test/fivequbit_tests/lin5qubit/",'linear','qregress/images/Function_Fitting/fivequbit/')
plot("function-calc-test/fivequbit_tests/quad5qubit/",'quadratic','qregress/images/Function_Fitting/fivequbit/')
plot("function-calc-test/fivequbit_tests/sin5qubit/",'sine','qregress/images/Function_Fitting/fivequbit/')

In [ ]:
plot("BSE-calc-test/5qubit/",'BSE','qregress/images/BSE/fivequbit/')
plot("BSE-calc-test/16qubit/",'BSE','qregress/images/BSE/sixteenqubit/')


In [ ]:
plot("qml_DDCC/5/",'DDCC','qregress/images/DDCC/')